# Module Test Template

## Module & Test Description

This Notebook is used to evaulate c and z (ratio of steel_strain to yield_strain) relationships. Positive z values values signify the steel is in compression; negative values correspond to tension.

c ranges from 0 (pure tension) to infinite (pure compression).

### Imports
##### General Imports

In [1]:
import os, sys, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import shapely as sh

##### Extend PYPATH to current folder:
This allows importing libraries from the same folder; <code>pathlib.Path().resolve()</code> returns the path of the current directory.

In [2]:
sys.path.extend([pathlib.Path().resolve()])

Import specific testing modules:

In [3]:
import rccolumn
import rcmaterials
import rcutilities

rcmaterials <version 0.0.6> successfully imported
rccolumn <version 0.0.4> successfully imported
rcutilities <version 0.0.1> successfully imported


## Input Values for Test Column Section

In [4]:
c_comp = rccolumn.CCOMP
c_tens = rccolumn.CTENS

conc = rcmaterials.ConcreteMaterial(5000)
bw = 16
h = 16

rebar = rcmaterials.RebarMaterial(60000)
layer_distances = np.array([2.5, 13.5])
layer_bar_sizes = np.array([9, 9])
layer_bar_counts = np.array([4, 4])

In [5]:
# Create function for returning array with total layer area by layer:
def get_layer_areas(layer_bar_sizes, layer_bar_counts, rebar: rcmaterials.RebarMaterial):
    layers = layer_bar_counts.shape[0]
    layer_areas = np.zeros(layers)
    for i in range(layers):
        layer_areas[i] = rebar.bar_areas[layer_bar_sizes[i]] * layer_bar_counts[i]
    return layer_areas

In [6]:
layer_areas = get_layer_areas(layer_bar_sizes, layer_bar_counts, rebar)
layer_areas

array([4., 4.])

<span style="color: orange;">**TODO**:</span> In the US you'll generally use the same grade of bar everywhere, but in Japan, there are three common grades. As a code enchancement, need to expand the ability to have different layer fy and ey properties.

In [7]:
# Calculate other attributes of column, determined from factors above:

In [8]:
d = max(layer_distances)

Ag = bw * h
Astl = sum(layer_areas)
rho = Astl / Ag

Ag, Astl, rho

(256, 8.0, 0.03125)

<span style="color: orange;">**TODO**:</span> Create function to calculate the column *rho*.

<span style="color: orange;">**TODO**:</span> Create function to return d and d'

## Test Code at Pure Axial and Pure Moment Points
Compare the following simple functions to the *pm_points()* output (all four are from *rccolumn.py*)

In [9]:
# Pure Compression
Po = rccolumn.Po(bw * h, layer_areas, conc, rebar)
Po / 1000

1534.0

In [10]:
# Compare with get_pm
P, M = rccolumn.pm_points(c_comp, bw, h, layer_distances, layer_areas, conc, rebar)
P / 1000, M / 12000

(1534.0, 0.0)

In [11]:
# Capped Maximum Compression --- there is no comparison for this.
Pn =  rccolumn.Pn(bw * h, layer_areas, conc, rebar)
Pn / 1000

1227.2

In [12]:
# Pure Tension
Pnt = rccolumn.Pnt(bw * h, layer_areas, conc, rebar)
Pnt / 1000

-480.0

In [13]:
# Compare with get_pm
P, M = rccolumn.pm_points(c_tens, bw, h, layer_distances, layer_areas, conc, rebar)
P / 1000, M / 12000

(-480.0, 0.0)

In [14]:
# Pure Moment Case
# First need to calculate the Z ratio, then calculate c from Z:
z_m = rccolumn.z_at_pure_m(bw, h, layer_distances, layer_areas, conc, rebar)
c_m = rccolumn.c_from_z(z_m, d, conc, rebar)

# Next we can calculate the P and M
P, M = rccolumn.pm_points(c_m, bw, h, layer_distances, layer_areas, conc, rebar)

round(P / 1000, 3), round(M / 12000, 3)

(-0.0, 237.68)

<span style="color: dodgerblue;">**CONCLUSION**:</span> The above functions are working as planned.

## Investigate range of c values between *c<sub>max</sub>* and *Z* = 0
On a PM diagram, strain has a non-linear relationship with the rotation angle of a line segment from the origin to the corresponding point on the PM curve. Near c -> inf for pure compression; the strain is limited to concrete.ecu. We'll probably need a geometric distribution of c values in this range to get a reasonable distribution of PM points.

In [24]:
# One option is to traverse valuse from c_max down to c at z = 0:
c_max = rccolumn.CCOMP
c_at_Z_0 = rccolumn.c_from_z(z=0, layer_distance=d, concrete=conc, rebar=rebar)
c_max, c_at_Z_0

(inf, 13.5)

In [26]:
# Another option is to stay in the z-domain
z_max = rccolumn.max_z(conc, rebar)
z_max, 0

(1.45, 0)